<a href="https://colab.research.google.com/github/PaulVanDev/HoeffdingD/blob/master/EfficientHoeffdingD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
from scipy.signal import decimate
import math
import time
from sklearn.preprocessing import KBinsDiscretizer

# Direct transcription of Matlab code / Used as reference

References:
matlab code

https://stackoverflow.com/questions/9270496/ideas-for-gpu-implementation-of-hoeffdings-d-dependence-coefficient/9322657#9322657

HoeffdingD formule

http://support.sas.com/documentation/cdl/en/procstat/63104/HTML/default/viewer.htm#procstat_corr_sect016.htm

In [0]:
# Direct transcription of R code / Used as reference
def hoeffdingsD( x, y ):

    #N = size(x,1);
    N=x.shape
    #R = tiedrank( x );
    R=rankdata(x)
    #S = tiedrank( y );
    S=rankdata(y)

    #Q = zeros(N,1);
    Q=np.zeros(N[0])
    #parfor i = 1:N
    for i in range(0, N[0]):
        #Q[i] = 1 + sum( R < R[i] & S < S[i] );
        Q[i] = 1 + np.sum(np.bitwise_and(R<R[i] ,S<S[i]))
        #% and deal with cases where one or both values are ties, which contribute less
        #Q[i] = Q[i] + 1/4 * (sum( R == R[i] & S == S[i] ) - 1); #% both indices tie.  -1 because we know point i matches
        Q[i] = Q[i] + 1/4 * (np.sum(np.bitwise_and(np.isin(R,R[i]),np.isin(S,S[i])))-1)
        #Q[i] = Q[i] + 1/2 * sum( R == R[i] & S < S[i] ); #% one index ties.
        Q[i] = Q[i] + 1/2 * (np.sum(np.bitwise_and(np.isin(R,R[i]),S<S[i])))
        #Q[i] = Q[i] + 1/2 * sum( R < R[i] & S == S[i] ); #% one index ties.
        Q[i] = Q[i] + 1/2 * (np.sum(np.bitwise_and(R<R[i],np.isin(S,S[i])))) 
    #D1 = sum( (Q-1).*(Q-2) );
    #D2 = sum( (R-1).*(R-2).*(S-1).*(S-2) );
    #D3 = sum( (R-2).*(S-2).*(Q-1) );
    D1 = np.sum( np.multiply((Q-1),(Q-2)) );
    D2 = np.sum( np.multiply(np.multiply((R-1),(R-2)),np.multiply((S-1),(S-2)) ) );
    D3 = np.sum( np.multiply(np.multiply((R-2),(S-2)),(Q-1)) );


    D = 30*((N[0]-2)*(N[0]-3)*D1 + D2 - 2*(N[0]-2)*D3) / (N[0]*(N[0]-1)*(N[0]-2)*(N[0]-3)*(N[0]-4));
    #p=(N[0]-1)*D*math.pow(math.pi, 4)/60+math.pow(math.pi, 4)/72

    return D

# My best python implementation of HoeffdingD

In [0]:
# Best Python implementation

def efficienthoeffdingsD( xin, yin ):
  
    #crop data to the smallest array, length have to be equal
    if len(xin)<len(yin):
      yin=yin[:len(xin)]
    if len(xin)>len(yin):
      xin=xin[:len(yin)]
      
    # dropna
    x = xin[~(np.isnan(xin) | np.isnan(yin))]
    y = yin[~(np.isnan(xin) | np.isnan(yin))]

    # undersampling if length too long
    lenx=len(x)
    if lenx>100000:
        factor=math.ceil(lenx/100000)
        x=x[::factor]
        y=y[::factor]
    
    # bining if too much "definition"
    if len(np.unique(x))>50:
        est = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='uniform')
        est.fit(x.reshape(-1, 1))  
        Rtemp = est.transform(x.reshape(-1, 1))
        R=rankdata(Rtemp)
    else:
        R=rankdata(x)
    if len(np.unique(y))>50:
        est1 = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='uniform')
        est1.fit(y.reshape(-1, 1))  
        Stemp = est1.transform(y.reshape(-1, 1))
        S=rankdata(Stemp)
    else:
        S=rankdata(y)      
      
    # core processing
    N=x.shape
    dico={(np.nan,np.nan):np.nan}
    dicoRin={np.nan:np.nan}
    dicoSin={np.nan:np.nan}
    dicoRless={np.nan:np.nan}
    dicoSless={np.nan:np.nan}
    Q=np.ones(N[0])

    i=0;
    for r,s in np.nditer([R,S]):
        r=float(r)
        s=float(s)
        if (r,s) in dico.keys():
            Q[i]=dico[(r,s)]
        else:
          if r in dicoRin.keys():
              isinR=dicoRin[r]
              lessR=dicoRless[r]
          else:
              isinR=np.isin(R,r)
              dicoRin[r]=isinR
              lessR=np.less(R,r)
              dicoRless[r]=lessR
              
          if s in dicoSin.keys():
              isinS=dicoSin[s]
              lessS=dicoSless[s]
          else:
              isinS=np.isin(S,s)
              dicoSin[s]=isinS
              lessS=np.less(S,s)
              dicoSless[s]=lessS


          Q[i] = Q[i] + np.count_nonzero(lessR & lessS) \
                + 1/4 * (np.count_nonzero(isinR & isinS)-1) \
                + 1/2 * (np.count_nonzero(isinR & lessS)) \
                 + 1/2 * (np.count_nonzero(lessR & isinS)) 
          dico[(r,s)]=Q[i]
        i+=1
    
    D1 = np.sum( np.multiply((Q-1),(Q-2)) );
    D2 = np.sum( np.multiply(np.multiply((R-1),(R-2)),np.multiply((S-1),(S-2)) ) );
    D3 = np.sum( np.multiply(np.multiply((R-2),(S-2)),(Q-1)) );

    D = 30*((N[0]-2)*(N[0]-3)*D1 + D2 - 2*(N[0]-2)*D3) / (N[0]*(N[0]-1)*(N[0]-2)*(N[0]-3)*(N[0]-4));

    return D

In [35]:
x=np.random.randint(100, size=1000000)
y=(np.random.randint(100, size=1000000)*x)#^6)^2+x*x


df=pd.DataFrame(x,columns=['x'])
df['y']=y
#%timeit efficienthoeffdingsD_cython( x, y )
#efficienthoeffdingsD( x, y )
#efficienthoeffdingsD2( x, y )
#efficienthoeffdingsD_set( x, y )
#test1=efficienthoeffdingsD_set4( x, y )
#%timeit test1=df['x'].corr(df['y'])
#print('pearson:',test1)
#test3=np.correlate(x, y, mode='valid')

#%timeit efficienthoeffdingsD_set5( x, y )
#%timeit efficienthoeffdingsD_set6( x, y )
%timeit efficienthoeffdingsD_set8( x, y )
#print(efficienthoeffdingsD_set6( x, y ))
print(efficienthoeffdingsD_set8( x, y ))
%timeit test3=df.corr('spearman')
print('spearman:',df.corr('spearman'))
%timeit test3=df.corr('kendall')
print('kendall:',df.corr('kendall'))
#efficienthoeffdingsD_numba( x, y )
#test2=efficienthoeffdingsD_set2( x, y )

%timeit efficienthoeffdingsD_candidate( x, y )
#print(efficienthoeffdingsD_set6( x, y ))
print(efficienthoeffdingsD_candidate( x, y ))
#%timeit hoeffdingsD( x, y )
#print(hoeffdingsD( x, y ))


1 loop, best of 3: 412 ms per loop
0.1892787451017596
1 loop, best of 3: 1.74 s per loop
spearman:           x         y
x  1.000000  0.665578
y  0.665578  1.000000
1 loop, best of 3: 301 ms per loop
kendall:           x         y
x  1.000000  0.503328
y  0.503328  1.000000
1 loop, best of 3: 328 ms per loop
0.1892787451017596


In [0]:
x = np.random.normal(0,1,300000)
noise = np.random.normal(0,1,300000)
#y=np.square(x)+noise/10
#y=np.sqrt(x)+noise/10
#y=np.square(np.clip(x, -0.5, -0.2))
y=noise

df=pd.DataFrame(x,columns=['x'])
df['y']=y
#%timeit efficienthoeffdingsD_cython( x, y )
#testref=efficienthoeffdingsD( x, y )
#print('hoeffding_ref:',testref)
#efficienthoeffdingsD2( x, y )
#efficienthoeffdingsD_set( x, y )
#test1=efficienthoeffdingsD_set4( x, y )
test1=df['x'].corr(df['y'])
print('pearson:',test1)
#test3=np.correlate(x, y, mode='valid')
test2=efficienthoeffdingsD_set6( x, y )
print('hoeffding:',test2)
test2=efficienthoeffdingsD_set8( x, y )
print('hoeffding:',test2)
test3=df.corr('spearman')
print('spearman:',test3.iloc[0]['y'])
test4=df.corr('kendall')
print('kendall:',test4.iloc[0]['y'])

pearson: 1.0878600144957437e-05


KeyboardInterrupt: ignored

In [0]:
x = np.array([10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5])
y1 = np.array([8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68])
y2 = np.array([9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74])
y3 = np.array([7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73])
x4 = np.array([8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8])
y4 = np.array([6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89])

df=pd.DataFrame(x,columns=['x'])
df['y1']=y1
df['y2']=y2
df['y3']=y3
df['x4']=x4
df['y4']=y4
#%timeit efficienthoeffdingsD_cython( x, y )
#testref=efficienthoeffdingsD( x, y )
#print('hoeffding_ref:',testref)
#efficienthoeffdingsD2( x, y )
#efficienthoeffdingsD_set( x, y )
#test1=efficienthoeffdingsD_set4( x, y )

print('pearson:',df['x'].corr(df['y1']),df['x'].corr(df['y2']),df['x'].corr(df['y3']),df['x4'].corr(df['y4']))
#test3=np.correlate(x, y, mode='valid')
efficienthoeffdingsD_set5( x, y1 )
print('hoeffding:',efficienthoeffdingsD_set5( x, y1 ),efficienthoeffdingsD_set5( x, y2 ),efficienthoeffdingsD_set5( x, y3 ),efficienthoeffdingsD_set5( x4, y4 ))
test3=df.corr('spearman')
print('spearman:',test3)
test4=df.corr('kendall')
print('kendall:',test4)
